In [ ]:
import PyQt5
%matplotlib qt
%matplotlib inline

import numpy as np
import pandas as pd 

import mne
from mne import io
from mne.stats import permutation_t_test
from mne.datasets import sample


### Load Data 

In [ ]:
# T-Testing in MNE: https://mne.tools/stable/auto_examples/stats/plot_sensor_permutation_test.html#sphx-glr-auto-examples-stats-plot-sensor-permutation-test-py
# Matlab: https://www.fieldtriptoolbox.org/workshop/oslo2019/statistics/

data = pd.read_csv('light.csv', header=None, usecols=[*range(0,150000)]) 





In [ ]:
ch1 = ['FEF','vlPFC','PPC','STG','RAntThal all Bipolars', 
            'RPostThal all Bipolars','LAntThal all Bipolars','LPostThal all Bipolars']

n_channels = 8
sfreq = 1000 

#row 2 is with EEG parameter
info2 = mne.create_info(ch_names = ch1, sfreq = sfreq, ch_types='eeg')
raw2 = mne.io.RawArray(data, info2)
raw2.info

#### Set Parameters

In [ ]:
picks1 = mne.pick_types(raw2.info, eeg=True, eog=False,
                       stim=False, exclude='bads')
tmin= -0.2
tmax= 0.5


events = mne.find_events(raw2, stim_channel='STG', shortest_event=1)
epochs = mne.Epochs(raw2,events, picks=picks1,
                    preload=True)


In [ ]:
data = epochs.get_data()
times = epochs.times


In [ ]:
temporal_mask = np.logical_and(0.04 <= times, times <= 0.06)
data = np.mean(data[:, :, temporal_mask], axis=2)


In [ ]:
#n_permutations = 50000
n_permutations = 10000
T0, p_values, H0 = permutation_t_test(data, n_permutations, n_jobs=1)


In [ ]:
significance = picks1[p_values <= 0.05]
significant_names = [raw2.ch_names[k] for k in significance]

In [ ]:
print ("T Test Obtained for multiple comparison")
print (H0) # T statistic obtained by permutations and t-max trick for multiple comparison
print("\n")
print ("T statistic for all variables")
print (T0) # T-statistic observed for all variables 
print("\n")
print ("P values for all tests")
print (p_values) # p values for all tests 